In [196]:
import requests
from bs4 import BeautifulSoup
import time
import random
import json
from http.client import RemoteDisconnected


In [198]:
urls = [
    # 'https://next.rikunabi.com/kw%91%E5%8DH/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%83h%83%89%83C%83o%81%255B/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%92%CA%90M+%8DH%8E%96/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%83%255C%83t%83g%83E%83F%83A+%8AJ%94%AD/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%96%F2%8D%DC%8Et/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%8E%F3%95t+%88%C4%93%E0/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%8A%C5%8C%EC%8Et/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%95%DB%88%E7%8Em/?keywordsearch=1&log_f=1'    
    ]
fileNames = [
    # 'carpentor',
    'driver',
    'electrician',
    'engineer',
    'elementaryTeacher',
    'secretary',
    'nurse',
    'childminder'
    ]

In [199]:
jobNumberFinder = 'rnn-pageNumber rnn-textXl'
jobTitleFinder = "rnn-linkText rnn-linkText--black"
jobDetailBaseUrl = "https://next.rikunabi.com"
jobDetailDescriptionFinder = 'rn3-companyOfferRecruitment__headText'

In [200]:
# 後で削除
import socks,socket
from urllib.request import urlopen
from fake_useragent import UserAgent

userAgent = UserAgent().random
print(userAgent)

Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.17 Safari/537.36


In [202]:
data = []
index = 0
dataIndex = 1
htmlNumber = 1
failedTopUrls = []
failedDetailUrls = []
for url in urls:
    try:
        socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr='127.0.0.1', port=9050)
        socket.socket = socks.socksocket
        headers = {'User-Agent': UserAgent().random}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        time.sleep(0.5)
        jobNumber = soup.find("span", class_=jobNumberFinder).text
        print("url:jobNumber:" + jobNumber)
    except RemoteDisconnected:
        print('url:' + url)
        failedTopUrls.append(url)
        time.sleep(0.5)
        continue
    while(True):
        if(htmlNumber == 1):
            titles = soup.find_all("a", class_=jobTitleFinder)
        else:
            try:
                newUrl = url.replace('(?<=\?)(.*)(?=\/)', 'crn' + str(htmlNumber) + '.html')
                headers = {'User-Agent': UserAgent().random}
                response = requests.get(newUrl, headers=headers)
                time.sleep(0.5)
                soup = BeautifulSoup(response.content, "html.parser")
                response.close()
                titles = soup.find_all("a", class_=jobTitleFinder)
            except RemoteDisconnected:
                print('url:' + url + 'htmlnumber=' + htmlNumber)
                failedDetailUrls.append(newUrl)
                htmlNumber = htmlNumber + 50
                if(htmlNumber - 1 >= int(jobNumber)):
                    break
                time.sleep(random.randint(2,7))
                continue
        for title in titles:
            try:
                headers = {'User-Agent': UserAgent().random}
                jobDetailResponse = requests.get(jobDetailBaseUrl + title['href'], headers=headers)
                time.sleep(0.5)
                jobDetailSoup = BeautifulSoup(jobDetailResponse.content, "html.parser")
                jobDetailResponse.close()
                jobDescription = None if jobDetailSoup.find("div", class_=jobDetailDescriptionFinder) is None else jobDetailSoup.find("div", class_=jobDetailDescriptionFinder).text                                                                                                                                  
            except RemoteDisconnected:
                print('url:' + url + 'htmlnumber=' + htmlNumber)
                continue
            if jobDescription is None:
                try:
                    headers = {'User-Agent': UserAgent().random}
                    jobMoreDetailResponse = requests.get(jobDetailBaseUrl + title['href'].replace('nx1', 'nx2'), headers=headers)
                    time.sleep(0.5)
                    jobDetailMoreSoup = BeautifulSoup(jobMoreDetailResponse.content, "html.parser")
                    jobMoreDetailResponse.close()                
                    jobDescription = jobDetailMoreSoup.find("div", class_=jobDetailDescriptionFinder).text
                except RemoteDisconnected:
                    print('url:' + url + 'htmlnumber=' + htmlNumber)
                    continue
            data.append({str(dataIndex) : {'title': title.text, 'description':jobDescription}})
            dataIndex = dataIndex + 1
            time.sleep(random.randint(2,7))
        htmlNumber = htmlNumber + 50
        if(htmlNumber - 1 >= int(jobNumber)):
            break
        time.sleep(random.randint(2,7))
    
    # export data into json file
    with open('./data/' + fileNames[index] + '.json', mode='w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)
    print(fileNames[index] + ':done')
    index = index + 1
    
# export Top Url data into json file
with open('./data/failedTopUrls.json', mode='w', encoding='utf-8') as file:
    json.dump(failedTopUrls, file, ensure_ascii=False, indent=2)
    
# export detailed Url data into json file
with open('./data/failedDetailUrls.json', mode='w', encoding='utf-8') as file:
    json.dump(failedDetailUrls, file, ensure_ascii=False, indent=2)

url:jobNumber:1073


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [203]:
print(data)


[{'1': {'title': '学歴・経験不問／近距離・収集車ドライバー、積込助手／定時退社', 'description': '\r\n                    【前向きにがんばる人、応援します！】\r ・正社員として働きたい\r・昼間の固定した時間で働きたい\r・引っ越しが伴う転勤がない方がいい\r・人の役に立つ仕事がしたい\rどんな動機でもいいんです。\r「これからがんばりたい！」\rそんな方を応援する企業です。\r 【男女ともに活躍できる環境です】\r 女性も多く活躍しています。\r収集ごみはそこまで重くはないですし\r（普段みなさんが出しているごみを想像すればわかります）\r慣れてくるとスピードもアップします。\r 「人と自然の新しい関係」を構築し、地球環境を保全する\rという企業理念のもと、大阪府下、兵庫県下で幅広く事業を展開している泉興業。\r市民生活になくてはならない仕事をしています。\r\n                '}}, {'2': {'title': '★コンビニ・薬局などへ生活必需品を届ける＜配送・ドライバー＞', 'description': '\r\n                    業界改革を目指し、社長を筆頭に数年かけて事業を拡大してきました。\r輸送コストの効率化や物流をまるごとコントロールするような\r提案を行い、メーカーとの信頼構築に努めています。\r 今後も単なる運送業にとどまることなく、\rメーカーの物流を一元管理するロジスティクス企業として\r成長していけるよう、増員募集が決定。\r\n                '}}, {'3': {'title': '軽貨物ドライバー(軽・2t車)未経験OK/月収30〜50万可', 'description': '\r\n                    創立24年の食品運送会社であるトランスポートジャコの新規事業として、\r軽貨物事業の新規スタッフを大募集いたします！\r\n                '}}, {'4': {'title': '＜長距離ナシ！構内限定！＞”働くクルマ”の大型特殊ドライバー', 'description': '\r\n                    ＜30名以上の大規模増員＞\r1942年に誕生し

In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
myDatabase = client['jobAdDB']
myCollection = myDatabase['carpentor']
myCollection.insert_many(data)

In [179]:
import socks,socket
from urllib.request import urlopen

socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr='127.0.0.1', port=9050)
socket.socket = socks.socksocket

html = requests.get('http://checkip.dyndns.com/').text
print(html)


<html><head><title>Current IP Check</title></head><body>Current IP Address: 94.230.208.148</body></html>

